In [ ]:
%pip install superlinked

In [1]:
import pandas as pd

/opt/homebrew/anaconda3/envs/hug_face/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [8]:
import os
data_path = os.path.abspath('..')+'/data/'
pokedex_df = pd.read_csv(data_path+"pokedex_updated.csv").drop(columns=["Unnamed: 0"])
pokedex_df['ability_1'].fillna('None', inplace=True)
pokedex_df['ability_2'].fillna('None', inplace=True)

In [9]:
pokedex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1025 non-null   int64  
 1   name            1025 non-null   object 
 2   color           1025 non-null   object 
 3   habitat         1025 non-null   object 
 4   shape           1025 non-null   object 
 5   poke_type       1025 non-null   object 
 6   capture_chance  1025 non-null   float64
 7   height          1025 non-null   int64  
 8   weight          1025 non-null   int64  
 9   ability_0       1025 non-null   object 
 10  ability_1       1025 non-null   object 
 11  ability_2       1025 non-null   object 
 12  sprite          1025 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 104.2+ KB


In [10]:
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import (
    String,
    Float,
)
from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.dsl.space.number_space import NumberSpace
from superlinked.framework.dsl.space.categorical_similarity_space import (
    CategoricalSimilaritySpace,
)
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.common.parser.dataframe_parser import DataFrameParser
from superlinked.framework.common.embedding.number_embedding import Mode
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.query.param import Param
from superlinked.framework.dsl.query.query import Query
from superlinked.framework.dsl.query.result import Result
from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
    InMemoryApp,
)

In [11]:
@schema
class PokeSchema:
    color: String
    habitat: String
    capture_chance: Float
    id: IdField

pokemon = PokeSchema()

In [12]:
categories = pokedex_df["color"].unique()
print(categories)
habitats = pokedex_df["habitat"].unique()
print(habitats)

['green' 'red' 'blue' 'white' 'brown' 'yellow' 'purple' 'pink' 'gray'
 'black']
['grassland' 'mountain' 'waters-edge' 'forest' 'rough-terrain' 'cave'
 'urban' 'sea' 'rare']


In [13]:
color_space = CategoricalSimilaritySpace(
    category_input=pokemon.color, categories=categories
)
habitat_space = CategoricalSimilaritySpace(
    category_input=pokemon.habitat, categories=habitats
)
capture_chance_space = NumberSpace(
    number=pokemon.capture_chance, min_value=0, max_value=1, mode=Mode.SIMILAR
)

poke_index = Index(
    spaces=[
        color_space,
        habitat_space,
        capture_chance_space,
    ]
)

In [14]:
df_parser = DataFrameParser(schema=pokemon)
source: InMemorySource = InMemorySource(pokemon, parser=df_parser)
executor: InMemoryExecutor = InMemoryExecutor(
    sources=[source],
    indices=[poke_index],
)
app: InMemoryApp = executor.run()

In [15]:
source.put([pokedex_df])

In [16]:
def get_results(result):
    ids = [int(entry.stored_object['id']) for entry in result.entries]
    res = pokedex_df[pokedex_df["id"].isin(ids)]
    return res

In [17]:
query = (
    Query(
        poke_index,
        weights={
            color_space: Param("color_weight"),
            habitat_space: Param("habitat_weight"),
            capture_chance_space: Param("capture_chance_weight"),
        },
    )
    .find(pokemon)
    .similar(color_space.category, Param("color"))  
    .similar(habitat_space.category, Param("habitat")) 
    .similar(capture_chance_space.number, Param("chance")) 
    .limit(Param("limit"))
)

In [18]:
pokedex_df.iloc[24]

id                                                               25
name                                                        pikachu
color                                                        yellow
habitat                                                      forest
shape                                                     quadruped
poke_type                                                  electric
capture_chance                                                 0.75
height                                                            4
weight                                                           60
ability_0                                                    static
ability_1                                             lightning-rod
ability_2                                                      None
sprite            https://raw.githubusercontent.com/PokeAPI/spri...
Name: 24, dtype: object

In [19]:
query_params = {
    "color_weight": 10,
    "habitat_weight": 0,
    "capture_chance_weight": 0,
    "color": "yellow",
    "habitat": "grassland",
    "chance": 0.75,
}

result = app.query(query, limit=5, **query_params)
get_results(result)

,id,name,color,habitat,shape,poke_type,capture_chance,height,weight,ability_0,ability_1,ability_2,sprite
13,14,kakuna,yellow,forest,squiggle,bug,0.47,6,100,shed-skin,None,None,https://raw.githubusercontent.com/PokeAPI/spri...
14,15,beedrill,yellow,forest,bug-wings,bug,0.18,10,295,swarm,sniper,None,https://raw.githubusercontent.com/PokeAPI/spri...
24,25,pikachu,yellow,forest,quadruped,electric,0.75,4,60,static,lightning-rod,None,https://raw.githubusercontent.com/PokeAPI/spri...
25,26,raichu,yellow,forest,upright,electric,0.29,8,300,static,lightning-rod,None,https://raw.githubusercontent.com/PokeAPI/spri...
26,27,sandshrew,yellow,rough-terrain,upright,ground,1.00,6,120,sand-veil,sand-rush,None,https://raw.githubusercontent.com/PokeAPI/spri...
